## Import Libraries

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import HuggingFaceHub
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec
from langchain import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
from langchain_openai import ChatOpenAI
import streamlit as st
import json

c:\Python311\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Load and split documents

In [3]:
loader = TextLoader('../Materials/CarsInformation.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1010, which is longer than the specified 1000
Created a chunk of size 1031, which is longer than the specified 1000
Created a chunk of size 1058, which is longer than the specified 1000
Created a chunk of size 1048, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000


Created a chunk of size 1004, which is longer than the specified 1000
Created a chunk of size 1118, which is longer than the specified 1000
Created a chunk of size 1167, which is longer than the specified 1000
Created a chunk of size 1084, which is longer than the specified 1000
Created a chunk of size 1054, which is longer than the specified 1000
Created a chunk of size 1005, which is longer than the specified 1000
Created a chunk of size 1179, which is longer than the specified 1000
Created a chunk of size 1102, which is longer than the specified 1000
Created a chunk of size 1108, which is longer than the specified 1000
Created a chunk of size 1131, which is longer than the specified 1000
Created a chunk of size 1030, which is longer than the specified 1000
Created a chunk of size 1039, which is longer than the specified 1000
Created a chunk of size 1037, which is longer than the specified 1000
Created a chunk of size 1150, which is longer than the specified 1000
Created a chunk of s

## Initialize embeddings

In [4]:
embeddings = HuggingFaceEmbeddings()

c:\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


## Initialize Pinecone instance

In [5]:
pc = Pinecone(api_key= os.getenv('PINECONE_API_KEY'))

index_name = "langchain-demo"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )            
    )
index = pc.Index(index_name)
docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

## Load fine-tuned model ID

In [17]:
with open('../Fine-Tune/fine_tuned_model_id.txt', "r") as f:
    fine_tuned_model_id = f.read().strip()

print(fine_tuned_model_id)

ft:gpt-4o-mini-2024-07-18:personal::9wGN95V0


## Initialize ChatOpenAI

In [18]:
model_name = fine_tuned_model_id
llm = ChatOpenAI(model_name=model_name, organization='org-G8UtpAEtkeLatwCgEhQGaPOw')

## Define prompt template

In [19]:
template = """
    You are an advanced car shopping assistant chatbot. Your goal is to help users find the perfect car based on their preferences and requirements. You will ask for user preferences step-by-step and use the information provided to give personalized recommendations. Start by introducing yourself and then proceed with the following steps:

    1. Greet the user and introduce yourself.
    2. Ask the user for their budget range.
    3. Ask about the type of car they are interested in (e.g., sedan, SUV, truck, etc.).
    4. Inquire about the preferred brand or any specific brands they have in mind.
    5. Ask about the primary use of the car (e.g., daily commute, family trips, off-roading, etc.).
    6. Determine any must-have features (e.g., fuel efficiency, safety features, technology, etc.).
    7. Ask if they have any other specific requirements or preferences.
    8. Summarize the user's preferences and provide a few car recommendations based on their answers with the link of company website that can buy the car.
    9. Offer to provide more details about any of the recommended cars or answer any additional questions.

    Use the user's answers and the following combined context to answer the question accurately and concisely. If the user is unsure about something, offer suggestions or ask follow-up questions to help them clarify their needs.

    If the combined context does not contain the answer, simply state that you don't know.

    Combined Context: {context}
    Question: {question}
    Answer:
"""

## RAG Pipeline

In [20]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

rag_chain = RetrievalQA.from_chain_type(
    llm, retriever=docsearch.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

## List of queries for evaluation

In [21]:
queries = [
    "What’s the best electric car for long-distance travel?",
    "Can you suggest a reliable SUV for a family of four?",
    "What’s a good car with a spacious trunk?",
    "Which hybrid car offers the best fuel efficiency?",
    "Can you recommend a compact car that’s easy to park in the city?"
]

## Function to generate context retrieval and answer

In [22]:
def generate_evaluation_data(query):
    retrieval_results = docsearch.similarity_search(query)
    contexts = [result.page_content for result in retrieval_results]
    # combined_context = " ".join(contexts)
    result = rag_chain.invoke(query)
    return {
        "query": query,
        "contexts": contexts,
        "answer": result["result"]
    }

## Generate and store evaluation data

In [23]:
evaluation_data = [generate_evaluation_data(query) for query in queries]

In [24]:
print("Query: " + evaluation_data[0]['query'] + "\n")
print("Context: " + evaluation_data[0]['contexts'][0] + "\n")
print("Answer: " + evaluation_data[0]['answer'])


Query: What’s the best electric car for long-distance travel?

Context: 1. **Model Name**: Kia EV6
2. **Year**: 2024
3. **Starting Price**: CAD ~$50,000 (prices may vary)
4. **Engine Options**: 
   - Single-motor RWD (167-225 hp)
   - Dual-motor AWD (320 hp)
5. **Key Features**: 
   - Fast charging capability (up to 80% in 18 minutes)
   - Long-range battery (up to 500 km range)
   - 12.3-inch dual widescreen displays
   - Advanced Driver Assistance Systems (ADAS)
6. **Available Trims**: 
   - Standard Range RWD
   - Long Range RWD
   - Long Range AWD
   - GT-Line AWD
7. **Fuel Economy (City/Highway)**: Electric consumption ~18.5/17.8 kWh/100 km
8. **Safety Ratings**: Expected to be high; not yet rated by IIHS and NHTSA
9. **Warranty Information**: 
   - 5-year/100,000 km comprehensive warranty
   - 5-year/unlimited km roadside assistance
   - 8-year/160,000 km battery warranty
10. **Financial Rate**: Approx. 1.99% APR (subject to change)
11. **Lease Rate**: Approx. 2.49% (subject to c

## Save evaluation data to a JSON file for human evaluation

In [25]:
with open('evaluation_data.json', 'w') as f:
    json.dump(evaluation_data, f, indent=4)

print("Evaluation data has been generated and saved to evaluation_data.json")

Evaluation data has been generated and saved to evaluation_data.json


## Evaluation function for context relevance

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_context_relevance(questions, retrieved_contexts):
    vectorizer = TfidfVectorizer().fit_transform(retrieved_contexts + questions)
    vectors = vectorizer.toarray()
    relevance_scores = []
    
    for i, question in enumerate(questions):
        context_vector = vectors[i]
        question_vector = vectors[len(questions) + i]
        cosine_sim = cosine_similarity([context_vector], [question_vector])[0][0]
        relevance_scores.append(cosine_sim)
    
    return relevance_scores

## Retrieve context and generate answers

In [27]:
retrieved_contexts = []

for data in evaluation_data:
    retrieved_contexts.append(data['contexts'][0])

generated_answers = []

for data in evaluation_data:
    generated_answers.append(data['answer'])

## Evaluate context relevance

In [28]:
context_relevance_scores = evaluate_context_relevance(queries, retrieved_contexts)

context_relevance_scores

[0.10596395744140033,
 0.020952840189126704,
 0.0,
 0.1951946798984531,
 0.03682295724389945]

## Display results

In [30]:
evaluation_results = {
    "questions": queries,
    "retrieved_contexts": retrieved_contexts,
    "generated_answers": generated_answers,
    "context_relevance_scores": context_relevance_scores
}

print(json.dumps(evaluation_results, indent=4))

with open('evaluation_data.json', 'w') as f:
    json.dump(evaluation_results, f, indent=4)

print("Evaluation data has been generated and saved to evaluation_data.json")

{
    "questions": [
        "What\u2019s the best electric car for long-distance travel?",
        "Can you suggest a reliable SUV for a family of four?",
        "What\u2019s a good car with a spacious trunk?",
        "Which hybrid car offers the best fuel efficiency?",
        "Can you recommend a compact car that\u2019s easy to park in the city?"
    ],
    "retrieved_contexts": [
        "1. **Model Name**: Kia EV6\n2. **Year**: 2024\n3. **Starting Price**: CAD ~$50,000 (prices may vary)\n4. **Engine Options**: \n   - Single-motor RWD (167-225 hp)\n   - Dual-motor AWD (320 hp)\n5. **Key Features**: \n   - Fast charging capability (up to 80% in 18 minutes)\n   - Long-range battery (up to 500 km range)\n   - 12.3-inch dual widescreen displays\n   - Advanced Driver Assistance Systems (ADAS)\n6. **Available Trims**: \n   - Standard Range RWD\n   - Long Range RWD\n   - Long Range AWD\n   - GT-Line AWD\n7. **Fuel Economy (City/Highway)**: Electric consumption ~18.5/17.8 kWh/100 km\n8. 